In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

2022-05-31 17:13:03 Configured OSMnx 1.1.1
2022-05-31 17:13:03 HTTP response caching is on


In [2]:
df = pd.read_json("/home/jovyan/work/scripts/areas.json")

In [3]:
for c in df.columns.unique():
        aup.log(f"\n Starting municipality filters for {c}")
        # Creates empty GeoDataFrame to store specified municipality bike infrastructure
        fm = gpd.GeoDataFrame()
        # Iterates over municipality codes for each metropolitan area or capital
        for i in range(len(df.loc["mpos", c])):
            # Extracts specific municipality code
            m = df.loc["mpos", c][i]
            # Downloads municipality polygon according to code
            query = f"SELECT * FROM censo_urbano.censo_fm_2020 WHERE \"CVEGEO\" LIKE \'{m}\'"
            fm = fm.append(aup.gdf_from_query(query, geometry_col='geometry'))
            aup.log(f"Downloaded {m} GeoDataFrame at: {c}")

In [ ]:
fm = fm.set_crs("EPSG:4326")

In [45]:
ciclovia = fm[fm['CICLOVIA']==1]
ciclocarril = fm[fm['CICLOCARRIL']==1]

In [46]:
bike_infra = ciclovia.append(ciclocarril)

In [47]:
bike_infra = bike_infra.to_crs("EPSG:6372")
centroid = bike_infra.centroid
bike_infra = bike_infra.to_crs("EPSG:4326")
centroid = centroid.to_crs("EPSG:4326")

In [48]:
bike_infra['centroid'] = centroid

In [52]:
bike_points = bike_infra[['CVEGEO']]
bike_points['geometry'] = centroid.copy()

/tmp/ipykernel_556/4180384012.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bike_points['geometry'] = centroid.copy()


In [56]:
bike_points= gpd.GeoDataFrame(bike_points)
bike_points = bike_points.set_crs("EPSG:4326")